In [ ]:
import time
import sys

sys.path.append('/home/cjw/Code/Unet')
#sys.path.append('/media/cjw/PythonLib/Unet')
from skimage.io import imread
from skimage.transform import resize
from matplotlib import pyplot as plt
import numpy as np
from scipy.ndimage.morphology import distance_transform_edt, binary_dilation
from dnet import dnet2d

%matplotlib inline

In [ ]:
import tensorflow as tf

In [ ]:
params = dict()

params['width'] = 512
params['height'] = 512
params['nchannels'] = 1
params['channels'] = [0]
params['nepochs'] = [1]
params['batchsize'] = [512]
params['learning_rate'] = 0.001
params['restore'] = False
params['latent_size'] = 1
params['net_sizes'] = [[32, 1, 1], [32, 3, 1], [32, 3, 2],
                       [32, 3, 4], [32, 3, 6], [32, 3, 4], [32, 3, 2], [2, 1, 1]]
params['droprate'] = 0.1
params['stdev'] = 0.04

In [ ]:
all_data_tif = imread('/ssd1/cjw/Data/EM/Training.tif_RotShift.tif')
all_data_tif.shape

In [ ]:
    
## read in the data

data_tif = all_data_tif[:,0, :, :]
labels_tif = all_data_tif[:,1, :, :]
labels_tif = binary_dilation(labels_tif).astype(labels_tif.dtype)
#labels_tif = binary_dilation(labels_tif).astype(labels_tif.dtype)
print(labels_tif.shape)

#create the background channel for the labels
labshape = labels_tif.shape

temp = 0*labels_tif
labels_tif = np.stack([labels_tif, temp], axis=-1)
print(labels_tif.shape)

#normalize images 0-1
#try normalizing wit zero mean and equal sdev

dmin = data_tif.min(axis=(1,2),keepdims=True)
dmax = data_tif.max(axis=(1,2),keepdims=True)
dmean = data_tif.mean(axis=(1,2),keepdims=True)
dstd = data_tif.std(axis=(1,2),keepdims=True)

#data = (data_tif - dmin)/(dmax - dmin)
data = (data_tif - dmean)/dstd
data = np.expand_dims(data, -1)
labels = (labels_tif - labels_tif.min())/(labels_tif.max() - labels_tif.min())
labels[:,:,:,-1] = (labels[:,:,:, 0] < .5).astype(np.float32)
#put cells with both channels into channel 0
# lsum = labels.sum(axis=(-1))
# s2 = lsum > 1
# s2 = s2.astype(np.float32)
# labels[:,:,:,1] -= s2 
data_tif.shape, data.shape, labels.shape, labels_tif.shape, labels.max(axis=(0,1,2))

In [ ]:
plt.imshow(labels[0,:,:,1])
(labels[:,:,:,1] > .5).dtype

In [ ]:

tf.reset_default_graph()
u = dnet2d.dnet2d(params)
u.x = data
u.y = labels

In [ ]:
w = 512

In [ ]:
tf.reset_default_graph()
images = tf.placeholder(tf.float32, (None, w, w, params['nchannels']))
masks = tf.placeholder(tf.float32, (None, w, w, labels.shape[-1]))
learning_rate = tf.placeholder(tf.float32, ())
u.learning_rate = learning_rate

In [ ]:
enc = u.create_dnet(images, True)

In [ ]:
u.net

In [ ]:
#d = u.create_decoder()
u.create_loss(masks)
u.create_opt()

In [ ]:
u.x.shape, u.y.shape

In [ ]:
#u.set_validation(20)
u.xtrain = u.x[0:375,:,:]
u.ytrain = u.y[0:375,:,:]
u.xtest = u.x[375:,:,:]
u.ytest = u.y[375:,:,:]

In [ ]:
sess = tf.Session()

In [ ]:
# tf.summary.scalar('loss', u.loss)
# si = tf.slice(u.decoder, [5, 0, 0, 0, 0], [10,1,256,256, 1])

# si = si[:,:,:,:,0]
# print(si)
# tf.summary.image('res', si, max_outputs=1)    
# merged = tf.summary.merge_all()



# logdir = "logdir"
# logwriter = tf.summary.FileWriter(logdir, sess.graph)

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
u.stdev = 0.1
#u.learning_rate = 0.0001

In [ ]:

rate = 0.005
#loss_file = open('progress.dat', 'w', buffering=1)

tbx, tbm = u.get_simple_batch(16)
for i in range(2500):
    t1 = time.time()
    rate = np.maximum(rate*.9999999, 0.000005)
    bx, bm = u.get_simple_batch(16)
#     plt.figure(figsize=(2,2))
#     plt.imshow(bm[0])
    _, res, err = sess.run([u.opt, u.decoder_softmax, u.loss],
                           feed_dict={images:bx, masks:bm, learning_rate:rate})
    if i % 100 == 0:
        print(i, err, err, res[0,:,:,:].min(), res[0,:,:,:].max(), rate)
        nr = np.random.randint(0,16)
        #nr = np.argmin(res[:,:,:,4].sum(axis=(1,2)))
        print(nr)
        
        tres = sess.run(u.decoder_softmax,
                           feed_dict={images:tbx, learning_rate:rate})
        
        plt.figure(figsize=(24,12))
        plt.subplot(2,5,1)
        plt.imshow(res[nr,:,:,0])
        plt.subplot(2,5,2)
        plt.imshow(res[nr,:,:,1])
        plt.subplot(2,5,3)
        plt.imshow(bm[nr,:,:,0])
        plt.subplot(2,5,4)
        plt.imshow(bm[nr,:,:,1])
        plt.subplot(2,5,5)
        bxd = bx[nr,:,:,0]
        bxd = (bxd - bxd.min())/(bxd.max() - bxd.min())
        plt.imshow(bxd)
        
        plt.subplot(2,5,6)
        plt.imshow(tres[nr,:,:,0])
        plt.subplot(2,5,7)
        plt.imshow(tres[nr,:,:,1])
        plt.subplot(2,5,8)
        plt.imshow(tbm[nr,:,:,0])
        plt.subplot(2,5,9)
        plt.imshow(tbm[nr,:,:,1])
        plt.subplot(2,5,10)
        bxd = tbx[nr,:,:,0]
        bxd = (bxd - bxd.min())/(bxd.max() - bxd.min())
        plt.imshow(bxd)
        
    
#         plt.imshow(res[nr,:,:,5])
#         plt.subplot(2,5,7)
#         plt.imshow(res[nr,:,:,6])
#         plt.subplot(2,5,8)
#         plt.imshow(res[nr,:,:,7])
#         plt.subplot(2,5,9)
#         plt.imshow(res[nr,:,:,8])
#         plt.subplot(1,6,4)
#         plt.imshow(res[nr,:,:,1])
#         plt.subplot(1,6,5)
#         plt.imshow(bm[nr,:,:,1])
#         plt.subplot(1,6,6)
#         plt.imshow(bx[nr,:,:,1])
        plt.show()
        #plt.savefig("images/image{0:05d}".format(i))
    t2 = time.time()
    if i % 10 == 0:
        print(i, err, (t2 - t1), rate)


In [ ]:
rmx = res[2,:,:,0].max(axis=(0))
rmx.shape, rmx.max(), rmx.min()

In [ ]:
na = 10
plt.figure(figsize=(16,8))
plt.subplot(1,3,1)
print(tbm.shape)
plt.imshow((tres[na,:,:,0]))
plt.subplot(1,3,2)
plt.imshow(tbm[na,:,:,0])
plt.subplot(1,3,3)
plt.imshow(tbx[na,:,:,0])


In [ ]:
#test_tif = imread('/ssd1/cjw/Data/sez/RawTifs/16_hours_punched_overnight/NDExp_Point0008_Seq0008.tif')
test_tif = u.xtest
#test_tif = labels[-1,:,:,:] #imread('/ssd1/cjw/Data/sez/RawTifs/NDExp_Point0021_Seq0021.tif')
# ymax = test_tif.max(axis=(0,1), keepdims=True)
# ymin = test_tif.min(axis=(0,1), keepdims=True)
# ymean = test_tif.mean(axis=(0,1), keepdims=True)
# ystd = test_tif.std(axis=(0,1), keepdims=True)
#test_tif = (test_tif - ymin)/(ymax - ymin + 0.00001)
test_tif = (test_tif - ymean)/ystd
test_tif.shape, ymin.shape

vres0 = sess.run(u.decoder_softmax,
                feed_dict={images:u.xtest})

vres0.shape

In [ ]:
na = 10
plt.figure(figsize=(24,12))
plt.subplot(1,3,1)
plt.imshow(u.ytest[na, :,:,0])
plt.subplot(1,3,2)
plt.imshow(vres0[na,:,:,0])
plt.subplot(1,3,3)
plt.imshow(u.xtest[na,:,:,0])

In [ ]:
def make_batch(data, start = 0):
    tclist = list()
    print(start)
    ix = start
    jy = start
    for i in range(6):
        jy = start
        for j in range(6):
            p = data[jy:jy + 256, ix:ix + 256,:]
            p = np.expand_dims(p, 0)
            tclist.append(p)
            jy = jy + 256
            #print(i, j, ix, jy)
        ix += 256

    tc = np.concatenate(tclist, axis=0)
    return tc

tc0 = make_batch(test_tif,start=0)
tc1 = make_batch(test_tif, start=128)
tc0.shape, tc1.shape

In [ ]:
vres0 = sess.run(u.decoder_softmax,
                feed_dict={images:tc0})

vres1 = sess.run(u.decoder_softmax,
                feed_dict={images:tc1})
vres0.shape, vres1.shape

In [ ]:
def reconstruct(v1, w, nx, ny, offset=0, pad=0):
    p = np.zeros((ny*w + pad, nx*w + pad, 5), dtype=np.float32)

    ix = offset
    jy = offset
    index = 0
    for i in range(nx):
        jy = offset
        for j in range(ny):
            p[jy:jy + 256, ix:ix + 256, :] = v1[index]
            jy = jy + 256
            index += 1
            #print(i, j, ix, jy)
        ix += 256
    return p
r0 = reconstruct(vres0, 256, 6, 6, offset=0, pad=128)
r1 = reconstruct(vres1, 256, 6, 6, offset=128, pad = 128)

r0 = np.expand_dims(r0, 0)
r1 = np.expand_dims(r1, 0)
r = np.concatenate([r0, r1])
r  = r.max(axis=(0))
r.shape, r.max()

In [ ]:

plt.figure(figsize=(12,12))

# az = r*0
# az[:,:,0] = r[:,:,-1]
# az[:,:,1] = r[:,:,-1]
# az[:,:,2] = r[:,:,-1]
# az = r - az

#plt.subplot(1,3,1)
plt.imshow(r[:,:,0:3]) #0[0,150:350,100:300,1])

#plt.subplot(1,3,4)
#plt.imshow(r0[0]) #[0,150:350,100:300,1])

#plt.subplot(1,3,3)
#plt.imshow(r1[0]) #[150:350,100:300,1])

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(test_tif[0:6*256+128, 0:6*256+128, :])

In [ ]:
plt.imshow(tc0[0,:,:,0])

In [ ]:
import tifffile
r.shape

In [ ]:
tifffile.imsave('/ssd1/cjw/Data/sez/unetout0017-2.tif', np.moveaxis(r, -1, 0))

In [ ]:
from skimage.morphology import binary_erosion

blab = labels[2, :,:,1]
plt.subplot(1,2,1)
plt.imshow(blab)
plt.subplot(1,2,2)
plt.imshow(binary_erosion(binary_erosion(blab)))
labels.shape

In [ ]:
plt.imshow(data[6,:,:,0:3])

In [ ]:
!%mkdir '/ssd1/cjw/sez/unet_checkpoint'

In [ ]:
saver = tf.train.Saver()
saver.save(sess, '/ssd1/cjw/sez/Checkpoints/2019-02-04-zero-mean/unet_checkpoint', global_step=5000)